<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/images_load_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This a modified Google TF team notebook to demonstrate varios ways of loading images for further processing by neural networks.  As I found out, the preferred way of image using directory names as labels via tf.dataset doesn't work with tf.keras (neither it does with keras) bcs models built with tf.keras doesn't allow for string labels.  This problem has been first found in May and, allegedly fixed.  As I found out in June it wasn't...I found a workaround.  It allows to change labels into integers withouts a significant perfomance penalty (see melanoma_identification)

# Load images

This tutorial provides a simple example of how to load an image dataset using `tf.data`.

The dataset used in this example is distributed as directories of images, with one class of image per directory.

## Setup

In [ ]:
import tensorflow as tf

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
tf.__version__

### Retrieve the images

Before you start any training, you will need a set of images to teach the network about the new classes you want to recognize. You can use an archive of creative-commons licensed flower photos from Google.

Note: all images are licensed CC-BY, creators are listed in the `LICENSE.txt` file.

In [ ]:
import pathlib
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

After downloading (218MB), you should now have a copy of the flower photos available.

The directory contains 5 sub-directories, one per class:

In [ ]:
next(iter(data_dir.glob('*/*.jpg')))

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [ ]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

Each directory contains images of that type of flower. Here are some roses:

In [ ]:
roses = list(data_dir.glob('roses/*'))

for image_path in roses[:3]:
    display.display(Image.open(str(image_path)))

## Load using `keras.preprocessing`

A simple way to load images is to use `tf.keras.preprocessing`.

In [ ]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

Define some parameters for the loader:

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [ ]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

Inspect a batch:

In [ ]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(32):
      ax = plt.subplot(6,6,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [ ]:
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

## Load using `tf.data`

The above `keras.preprocessing` method is convienient, but has three downsides: 

1. It's slow. See the performance section below.
1. It lacks fine-grained control.
1. It is not well integrated with the rest of TensorFlow.

To load the files as a `tf.data.Dataset` first create a dataset of the file paths:

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [ ]:
for f in list_ds.take(5):
  print(f.numpy())

Write a short pure-tensorflow function that converts a file path to an `(img, label)` pair:

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] 

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

Use `Dataset.map` to create a dataset of `image, label` pairs:

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test = labeled_ds.take(2)

In [ ]:
for _, label in test:
  #print("Image shape: ", image)
  #print("Label: ", label.numpy())
  #label = tf.strings.to_number('42', tf.int32)
  print("Label: ", label.numpy())

In [ ]:
for image, label in test:
  print("Image shape: ", image.numpy())
  print("Label: ", label.numpy())
  #print("Label: ", label.numpy().decode('ascii'))

**EXPERIMENTS**

changing labels to ints in the above dataset

In [ ]:
# using "test" to experiment with
imgs = []
labels = []
for image, label in test:
  if label.numpy().decode('ascii') == 'daisy':
    label = np.array([0])
  else: 
    label = np.array([1])  
  imgs.append(image.numpy())
  labels.append(label)

changed_test = tf.data.Dataset.from_tensor_slices((tf.constant(imgs), tf.constant(labels)))

In [ ]:
for img, label in changed_test:
  print("Image shape: ", image.numpy())
  print("Label: ", label.numpy())

In [ ]:
#experiments:
arg = tf.convert_to_tensor(['1','0','1'], dtype=tf.string); arg

In [ ]:
def test_path(file_path):
  #label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img

In [ ]:
test_path('/root/.keras/datasets/flower_photos/dandelion/5607669502_ccd2a76668_n.jpg').numpy()

In [ ]:
test = pathlib.Path(data_dir/'roses/'); test

In [ ]:
image_count = len(list(test.glob('*.jpg')))
image_count

In [ ]:
# random array of ones and zeros
labels = [0]*400 + [1]*241

In [ ]:
#import shutil
ex = list(test.glob('*.jpg'))

In [ ]:
filenames = []
for item in ex:
    filenames.append(str(item))

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))

In [ ]:
# Function to load and preprocess each image
def _parse_fn(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img)
    img = (tf.cast(img, tf.float32)/127.5) - 1
    img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img, label

In [ ]:
IMAGE_SIZE = 224 # Minimum image size for use with MobileNetV2
BATCH_SIZE = 32
train_data = train_data.map(_parse_fn)

In [ ]:
for image, label in train_data.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

### Basic methods for training

To train a model with this dataset you will want the data:

* To be well shuffled.
* To be batched.
* Batches to be available as soon as possible.

These features can be easily added using the `tf.data` api.

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [ ]:
train_ds = prepare_for_training(labeled_ds)

#image_batch, label_batch = next(iter(train_ds))

In [ ]:
type(train_ds)

In [ ]:
test_ds = prepare_for_training(train_data)

# Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(test_ds,
          epochs=10,
          steps_per_epoch = 3600//320)

In [ ]:
model_multi = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(5, activation='softmax')
])

model_multi.summary()

In [ ]:
# Compile the model
model_multi.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds,
          epochs=5,
          steps_per_epoch = 3600//320)

## Performance

Note: This section just shows a couple of easy tricks that may help performance. For an in depth guide see [Input Pipeline Performance](../../guide/performance/datasets).

To investigate, first here's a function to check the performance of our datasets:

In [ ]:
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))

Let's compare the speed of the two data generators:

In [ ]:
# `keras.preprocessing`
timeit(train_data_gen)

In [ ]:
# `tf.data`
timeit(train_ds)

A large part of the performance gain comes from the use of `.cache`.

In [ ]:
uncached_ds = prepare_for_training(labeled_ds, cache=False)
timeit(uncached_ds)

If the dataset doesn't fit in memory use a cache file to maintain some of the advantages:

In [ ]:
filecache_ds = prepare_for_training(labeled_ds, cache="./flowers.tfcache")
timeit(filecache_ds)

# Sandbox

In [ ]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
#loading .npy & .npz files. I am using a set of processed medical images from the kaggle pulmonary fibrosis competition
import numpy as np
# load numpy array from npy file
from numpy import load
# load array
#images_set = load('data.npy')

# load dict of arrays
dict_data = load('data.npz')
# extract the first array
images_set = dict_data['arr_0']
len(images_set)

In [ ]:
import functools as f
import matplotlib.pyplot as plt
res_image = f.reduce(lambda x,y: np.add(x,y), images_set)/5
fig, axes = plt.subplots(2,3, figsize=(20,20))
axes[0,0].imshow(res_image, cmap='gray')
axes[0,1].imshow(images_set[0], cmap='gray')
axes[0,2].imshow(images_set[1], cmap='gray')
axes[1,0].imshow(images_set[2], cmap='gray')
axes[1,1].imshow(images_set[3], cmap='gray')
axes[1,2].imshow(images_set[4], cmap='gray')